# Understanding Memory in LLMs

In the previous Notebook 03, we successfully explored how OpenAI models can enhance the results from Azure Cognitive Search. [Bing Chat](http://chat.bing.com/) is a search engine with a GPT-4 model that utilizes the content of search results to provide context and deliver accurate responses to queries.

However, we have yet to discover how to engage in a conversation with the LLM. With Bing Chat, this is possible, as the LLM can understand and reference the previous responses.

There is a common misconception that GPT models have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

The aim of this Notebook is to demonstrate how we can "provide memory" to the LLM by utilizing prompts and context.

In [28]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferMemory
from openai.error import OpenAIError
from langchain.docstore.document import Document
from langchain.memory import CosmosDBChatMessageHistory

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from app.utils import (
    get_search_results,
    order_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    embed_docs,
    search_docs,
    get_answer,
)

from app.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_CHAT_PROMPT

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

# Don't mess with this unless you really know what you are doing
AZURE_SEARCH_API_VERSION = '2021-04-30-Preview'
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"


# Change these below with your own services credentials
AZURE_SEARCH_ENDPOINT ="ENTER YOUR VALUE"
AZURE_SEARCH_KEY = "ENTER YOUR VALUE"
AZURE_OPENAI_ENDPOINT = "ENTER YOUR VALUE"
AZURE_OPENAI_API_KEY = "ENTER YOUR VALUE"
AZURE_COSMOSDB_ENDPOINT = "ENTER YOUR VALUE"
AZURE_COSMOSDB_NAME = "ENTER YOUR VALUE"
AZURE_COSMOSDB_CONTAINER_NAME = "ENTER YOUR VALUE"
AZURE_COMOSDB_CONNECTION_STRING = "ENTER YOUR VALUE"

In [5]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_SEARCH_KEY"] = AZURE_SEARCH_KEY
os.environ["AZURE_SEARCH_ENDPOINT"] = AZURE_SEARCH_ENDPOINT
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_COSMOSDB_ENDPOINT"]=  AZURE_COSMOSDB_ENDPOINT
os.environ["AZURE_COSMOSDB_NAME"]=  AZURE_COSMOSDB_NAME
os.environ["AZURE_COSMOSDB_CONTAINER_NAME"]=  AZURE_COSMOSDB_CONTAINER_NAME
os.environ["AZURE_COMOSDB_CONNECTION_STRING"]=  AZURE_COMOSDB_CONNECTION_STRING

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [6]:
QUESTION = "Tell me some use cases for reinforcement learning?"
FOLLOW_UP_QUESTION = "Can you rephrase what you just said?"

In [7]:
# Define model
MODEL = "gpt-35-turbo"
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5)

In [8]:
# We create a very simple prompt template, just the question as is:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
# Let's see what the GPT model responds
response = chain.run(QUESTION)
printmd(response)

1. Game playing: Reinforcement learning has been successfully applied to game playing, such as AlphaGo and AlphaZero, which have defeated world champions in the games of Go, chess, and shogi.

2. Robotics: Reinforcement learning can be used to train robots to perform complex tasks, such as grasping objects, navigating through environments, and manipulating tools.

3. Autonomous vehicles: Reinforcement learning can be used to train autonomous vehicles to make decisions based on real-time data, such as traffic conditions, weather, and road obstacles.

4. Personalized recommendations: Reinforcement learning can be used to personalize recommendations for products, services, and content based on user behavior and preferences.

5. Finance: Reinforcement learning can be used to optimize investment portfolios, detect fraud, and predict market trends.

6. Healthcare: Reinforcement learning can be used to optimize treatment plans for patients with chronic conditions, such as diabetes and heart disease.

7. Advertising: Reinforcement learning can be used to optimize ad placement and targeting, based on user behavior and engagement.

8. Industrial automation: Reinforcement learning can be used to optimize manufacturing processes, reduce waste, and improve product quality.

In [10]:
#Now let's ask a follow up question
chain.run(FOLLOW_UP_QUESTION)

'As an AI language model, I cannot remember what I just said as I do not have the ability to retain memory. However, if you provide me with the specific sentence or phrase that you want me to rephrase, I will be happy to assist you.'

As you can see, it doesn't remember what it just responded. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [11]:
hist_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
                {history}
                Human: {question}
                AI:
            """
    )
chain = LLMChain(llm=llm, prompt=hist_prompt)

In [12]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response)

In [13]:
chain.run({"history":Conversation_history, "question": FOLLOW_UP_QUESTION})

'Sure, here are some examples of how reinforcement learning can be applied in various industries: game playing, robotics, autonomous vehicles, personalized recommendations, finance, healthcare, advertising, and industrial automation. For instance, reinforcement learning can be used to train robots to perform complex tasks, optimize investment portfolios, and personalize ad placement based on user behavior and engagement.'

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

In order to not duplicate code, we have put many of the code used in Notebook 3 into functions. These functions are in the app/utils.py and app/prompts.py files This way we can use these functios in the app that we will build later.

In [14]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

agg_search_results = get_search_results(QUESTION, indexes)
ordered_results = order_search_results(agg_search_results, reranker_threshold=1)

In [15]:
docs = []
for key,value in ordered_results.items():
    for page in value["chunks"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))

# Calculate number of tokens of our docs
tokens_limit = model_tokens_limit(MODEL)

if(len(docs)>0):
    num_tokens = num_tokens_from_docs(docs)
    print("Custom token limit for", MODEL, ":", tokens_limit)
    print("Combined docs tokens count:",num_tokens)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")


Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 80428


In [16]:
%%time
if num_tokens > tokens_limit:
    index = embed_docs(docs)
    top_docs = search_docs(index,QUESTION)
    
    # Now we need to recalculate the tokens count of the top results from similarity vector search
    # in order to select the chain type: stuff or map_reduce
    
    num_tokens = num_tokens_from_docs(top_docs)   
    print("Token count after similarity search:", num_tokens)
    chain_type = "map_reduce" if num_tokens > tokens_limit else "stuff"
    
else:
    # if total tokens is less than our limit, we don't need to vectorize and do similarity search
    top_docs = docs
    chain_type = "stuff"
    
print("Chain Type selected:", chain_type)

Number of chunks: 70


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token count after similarity search: 4693
Chain Type selected: map_reduce
CPU times: user 19.7 s, sys: 2.38 s, total: 22.1 s
Wall time: 12.5 s


In [17]:
# Get the answer
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type)
response['output_text']

'The documents provide information on techniques and theories related to reinforcement learning, but do not directly mention any specific use cases for reinforcement learning. However, with appropriate biases, complex reinforcement-learning problems can eventually be solvable, and the algorithms discussed in the documents can be used for reinforcement learning in various applications. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v1.pdf'

And if we ask the follow up question:

In [18]:
response = get_answer(docs=top_docs,  query=FOLLOW_UP_QUESTION, language="English",deployment=MODEL, chain_type=chain_type)
response['output_text']

'There is no information provided in the given content to answer the question.\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v1.pdf'

Until now we just have the same as the prior Notebook 03: results from Azure Search enhanced by OpenAI model, with no memory

**Now let's add memory to it:**

Reference: https://python.langchain.com/en/latest/modules/memory/examples/adding_memory_chain_multiple_inputs.html

In [19]:
# memory object, which is neccessary to track the inputs/outputs and hold a conversation.
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question")

response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning can be used in a variety of applications, such as robotics, game playing, and recommendation systems. For example, AlphaGo, a computer program developed by Google DeepMind, uses reinforcement learning to play the board game Go at a world-class level. In robotics, reinforcement learning can be used to teach robots how to perform complex tasks, such as grasping objects or navigating through environments. It can also be used in recommendation systems to optimize user engagement and satisfaction. However, the specific use cases for reinforcement learning depend on the problem domain and the available data. Sources: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v1.pdf.'

In [20]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

"Reinforcement learning has many use cases, including robotics, game playing, and recommendation systems. For example, Google DeepMind's AlphaGo program uses reinforcement learning to play Go at a world-class level. In robotics, reinforcement learning can be used to teach robots complex tasks, such as grasping objects or navigating through environments. It can also be used in recommendation systems to optimize user engagement and satisfaction. However, the specific use cases for reinforcement learning depend on the problem domain and available data. Additionally, there is a trade-off between exploration and exploitation in reinforcement learning, where the dilemma is between selecting actions to maximize the expected return according to the current world model and improving the world model to potentially achieve a higher expected return. This concept has been extensively studied, and optimal decision thresholds for the multi-armed bandit problem have been derived. These thresholds depe

In [21]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"I'm sorry, I couldn't find any relevant information in the provided sources to answer your question. Is there anything else I can help you with? Sources: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v1.pdf. \nSUGGESTION: Do you have any other questions or topics you would like to discuss?"

Let's check our memory to see that it's keeping the conversation

In [22]:
memory.buffer

"Human: Tell me some use cases for reinforcement learning?\nAI: Reinforcement learning can be used in a variety of applications, such as robotics, game playing, and recommendation systems. For example, AlphaGo, a computer program developed by Google DeepMind, uses reinforcement learning to play the board game Go at a world-class level. In robotics, reinforcement learning can be used to teach robots how to perform complex tasks, such as grasping objects or navigating through environments. It can also be used in recommendation systems to optimize user engagement and satisfaction. However, the specific use cases for reinforcement learning depend on the problem domain and the available data. Sources: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v1.pdf.\nHuman: Can you rephrase what you just said?\nAI: Reinforcement learning has many use cases, including robotics, game playing, and recommendation syst

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wisg to provide recommendations. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory, see [HERE](https://python.langchain.com/en/latest/_modules/langchain/memory/chat_message_histories/cosmos_db.html)

In [23]:
# Create CosmosDB instance from langchain cosmos class.

SESSION_ID = '001'  # this isstring value, normally you need to provide the user session id. As an example I have provided '001' as a session id
USER_ID = 'dummy01' # this is string value, normally you need to provide the user id. As an example I have provided 'dummy01' as a user id

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=AZURE_COSMOSDB_ENDPOINT,
    cosmos_database=AZURE_COSMOSDB_NAME,
    cosmos_container=AZURE_COSMOSDB_CONTAINER_NAME,
    connection_string=AZURE_COMOSDB_CONNECTION_STRING,
    session_id=SESSION_ID,
    user_id=USER_ID
    )

# prepare the cosmosdb instance
cosmos.prepare_cosmos()

In [30]:
# Create or Memory Object
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question",chat_memory=cosmos)

In [24]:
# Testing using our Question
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning has a variety of use cases such as game playing, robotics, recommendation systems, and autonomous driving. However, the specific use cases are not mentioned in the provided portion of the document. You can find more information on the topic in this source: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf.'

In [25]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

'I apologize for the confusion in my previous response. To rephrase, reinforcement learning can be used in various fields such as game playing, robotics, recommendation systems, and autonomous driving. However, the specific use cases are not mentioned in the provided portion of the document. If you would like to learn more about this topic, I recommend checking out this source: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf. Additionally, there is a trade-off between exploration and exploitation in reinforcement learning, where the dilemma is between selecting actions to maximize the expected return according to the current world model and improving the world model to potentially achieve a higher expected return. This concept has been studied extensively, with various approaches to finding optimal solutions for the multi-armed bandit problem. The optimal action selection mechanism will depend on the problem formulation, and the exploration/exploitation tradeoff 

In [26]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"You're welcome! Let me know if you have any other questions. \nSOURCES: N/A"

Let's check our Azure CosmosDB to see the whole conversation


In [27]:
#load message from cosmosdb
cosmos.load_messages()
cosmos.messages

[HumanMessage(content='Tell me some use cases for reinforcement learning?', additional_kwargs={}, example=False),
 AIMessage(content='Reinforcement learning has a variety of use cases such as game playing, robotics, recommendation systems, and autonomous driving. However, the specific use cases are not mentioned in the provided portion of the document. You can find more information on the topic in this source: https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf.', additional_kwargs={}, example=False),
 HumanMessage(content='Can you rephrase what you just said?', additional_kwargs={}, example=False),
 AIMessage(content='I apologize for the confusion in my previous response. To rephrase, reinforcement learning can be used in various fields such as game playing, robotics, recommendation systems, and autonomous driving. However, the specific use cases are not mentioned in the provided portion of the document. If you would like to learn more about this topic, I recommend 

![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, it struggles to respond to prompts like: Hello, Thank you, Bye, What's your name, What's the weather and any other task that is not search in the knowledge base.



# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

But, does this solve all the possible scenarios that a virtual assistant will require?  **What about if the answer to the Smart Search Engine is not related to text, but instead requires to look into tabular data?** The next notebook 05 explains and solves the tabular problem and the concept of Agents